In [1]:
import os, sys
import numpy as np


In [2]:
# our modules
from lace.cosmo import fit_linP
from lace.emulator import poly_p1d
from lace.archive import pnd_archive
from lace.archive import interface_archive
from lace.emulator.nn_emulator import NNEmulator
from lace.emulator.gp_emulator import GPEmulator
#from lace.emulator.emulator import P1D_emulator

/data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
emuparams = ['Delta2_p', 'n_p','mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']

# CREATE TRAINING AND TESTING ARCHIVE

In [5]:
archive = pnd_archive.archivePND(sim_suite="Cabayol23", z_max=4.5)
archive.get_training_data()
len(archive.training_data)

9900

In [6]:
archive_test = pnd_archive.archivePND(sim_suite="Cabayol23", pick_sim='central')
archive_test.get_testing_data()
len(archive_test.testing_data)

11

In [7]:
kMpc = archive_test.testing_data[0]['k_Mpc']
kMpc = kMpc[(kMpc>0) & (kMpc<4)]

## NEURAL NETWORK EMULATOR  

### The neural network emulator supposrts running both on GPU and CPU. No argument is needed, f the gpu is available, it will use it. Otherwise, the emulator trains on CPUs

### The neural network emulator uses the following default parameters:

- emu_params=['Delta2_p', 'n_p','mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']
- training_set=
- emulator_label=
- sims_label=
- nepochs=100
- step_size=75
- kmax_Mpc=4
- ndeg=5. For the extended version (kmax_Mpc=8), set to 7
- initial_weights=True. Always starts from the same iniial parameters. These were selected randomly. Set to False if random initialization is required.

### To create an emulator object, we can call NNemulator (to create a neural network emulator) or GP emulator (to create a Gaussian process emulator). 

### In both cases, we must always especify the sims_label to either 'Gadget0' or 'Nyx'.

### Example 1: We can train a custom emulator... 

#### A. passing a custom archive:

In [10]:
emulator = NNEmulator(archive=archive, nepochs=1)

Use custom archive provided by the user
Selected custom emulator


### or a training_set label

In [11]:
emulator = NNEmulator(training_set='Cabayol23',nepochs=1)

Selected training set from Cabayol23
Selected custom emulator


#### If none or both are provided, the emulator fails. 

### Example 2: We can train a pre defined emulator... 

#### A. with a training_set label

In [12]:
emulator = NNEmulator(training_set='Cabayol23', emulator_label='Cabayol23', nepochs=1)

Selected training set from Cabayol23
Select emulator in Cabayol23
Neural network emulating the optimal P1D of Gadget simulations fitting coefficients to a 5th degree polynomial. It goes to scales of 4Mpc^{-1} and z<4.5. The parameters passed to the emulator will be overwritten to match these ones


#### B. with an archive

In [13]:
emulator = NNEmulator(archive=archive,  emulator_label='Cabayol23', nepochs=1)

Use custom archive provided by the user
Select emulator in Cabayol23
Neural network emulating the optimal P1D of Gadget simulations fitting coefficients to a 5th degree polynomial. It goes to scales of 4Mpc^{-1} and z<4.5. The parameters passed to the emulator will be overwritten to match these ones


### Example 3: Load a pre-trained emulator, providing the path of the saved network parameters

In [8]:
emulator = NNEmulator(archive=archive,nepochs=1,model_path='NNmodels/NNEmulator_LaCEHC.pt', train=False)

Use custom archive provided by the user
Selected custom emulator
Model loaded. No training needed
The loaded model corresponds to the following emulation configuration:
:zmax: 4.5, kmax_Mpc: 4, ndeg: 5, emu_params: ['Delta2_p', 'n_p', 'mF', 'sigT_Mpc', 'gamma', 'kF_Mpc'], drop_sim: None), sims_training: Gadget


#### if the emulator parameters of the trained emulator are different to the arguments passed by the user, the code crushes:

In [9]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

## GAUSSIAN PROCESS EMULATOR

## The Gaussian process emulator uses the following default parameters:

- paramList=['Delta2_p', 'n_p','mF', 'sigT_Mpc', 'gamma', 'kF_Mpc']
- kmax_Mpc=10
- ndeg=4

In [17]:
archive = pnd_archive.archivePND(sim_suite="Pedersen21")
archive.get_training_data()
len(archive.training_data)

330

### Example 1: Train custom emulator 

#### with a defined training_set

In [18]:
emulator = GPEmulator(training_set='Pedersen21')

Selected training set from Pedersen21
Selected custom emulator
Training GP on 330 points


 /data/astro/scratch/lcabayol/anaconda3/envs/DESIenv6/lib/python3.10/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


GPs optimised in 0.69 seconds


#### with a custom archive

In [19]:
emulator = GPEmulator(archive=archive)

Use custom archive provided by the user
Selected custom emulator
Training GP on 330 points
GPs optimised in 0.68 seconds


#### if both are provided, the emulator fails

### Example 2: Pre-defined GP emulators:

In [20]:
emulator = GPEmulator(training_set='Pedersen21', emulator_label='Pedersen21')

Selected training set from Pedersen21
Select emulator in Pedersen21
Gaussian Process emulator predicting the P1D at each k-bin. It goes to scales of 3Mpc^{-1} and z<4.5. The parameters passed to the emulator will be overwritten to match these ones.
Training GP on 330 points
GPs optimised in 1.34 seconds


In [21]:
emulator = GPEmulator(training_set='Pedersen21', emulator_label='Pedersen23')

Selected training set from Pedersen21
Select emulator in Pedersen23
Gaussian Process emulator predicting the optimal P1Dfitting coefficients to a 5th degree polynomial. It goes to scales of 4Mpc^{-1} and z<4.5. The parameters passed to the emulator will be overwritten to match these ones
Training GP on 330 points
GPs optimised in 0.67 seconds


In [22]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

In [23]:
emulator = GPEmulator(training_set='Pedersen21')

Selected training set from Pedersen21
Selected custom emulator
Training GP on 330 points
GPs optimised in 0.69 seconds


In [24]:
p1d = emulator.emulate_p1d_Mpc(archive_test.testing_data[0],kMpc)

In [ ]:
import torch

In [ ]:
listdir = os.listdir('../lace/emulator/NNmodels_extended/')

In [ ]:
len(listdir)

30

In [ ]:
for modelname in listdir:
    number = modelname[:-3]
    
    # Extract the number
    number = number.split('_')[-1]
    print(number)
    if (int(number)!=99)|(int(number)!=222):
    
        emu = NNEmulator(archive=archive,ndeg=7,kmax_Mpc=4, model_path=f'NNmodels_extended/{modelname}', train=False)
        model=emu.emulator.state_dict()




        extra_info = {
            'zmax': '4.5',
            'kmax_Mpc': '8',
            'ndeg':7,
            'emu_params':["Delta2_p", "n_p", "mF", "sigT_Mpc", "gamma", "kF_Mpc"],
            'emu_type':"polyfit",
            'drop_sim':number,
            'sims_training':'Gadget'
        }

        # Create a dictionary to store the model and extra information
        data = {
            'model': model,
            'emulator_params': extra_info,

        }

        # Save the dictionary to a .pt file
        torch.save(data, f'/nfs/pic.es/user/l/lcabayol/DESI/LaCE/lace/emulator/NNmodels_extended/{modelname}')

8
Use custom archive provided by the user
Selected custom emulator
Model loaded. No training needed
The loaded model corresponds to the following emulation configuration:
:zmax: 4.5, kmax_Mpc: 4, ndeg: 5, emu_params: ['Delta2_p', 'n_p', 'mF', 'sigT_Mpc', 'gamma', 'kF_Mpc'], drop_sim': 8


ValueError: The arguments passed to the emulator do not match the loaded emulator configuration